## 1. Importing Necessary Libraries

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.stats import norm
from scipy.optimize import minimize_scalar
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

## 2. Data Collection and Preprocessing

In [2]:
def fetch_stock_data(symbol="RELIANCE.NS", period="2y"):
    """
    Fetch historical stock data from Yahoo Finance
    
    Parameters:
    - symbol: Stock symbol (default: RELIANCE.NS for Reliance Industries)
    - period: Time period for historical data
    
    Returns:
    - DataFrame with stock price data
    """
    print(f"\n1. Fetching data for {symbol}...")
    
    # Download stock data
    stock = yf.Ticker(symbol)
    data = stock.history(period=period)
    
    # Calculate additional metrics
    data['Returns'] = data['Close'].pct_change()
    data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
    
    # Remove NaN values
    data = data.dropna()
    
    print(f"   - Downloaded {len(data)} trading days of data")
    print(f"   - Current stock price: ₹{data['Close'].iloc[-1]:.2f}")
    print(f"   - Price range: ₹{data['Close'].min():.2f} - ₹{data['Close'].max():.2f}")
    
    return data

In [3]:
def calculate_volatility(data, method='historical'):
    """
    Calculate historical volatility using different methods
    
    Parameters:
    - data: Stock price DataFrame
    - method: Method for volatility calculation
    
    Returns:
    - Annualized volatility
    """
    if method == 'historical':
        # Standard deviation of log returns annualized
        daily_vol = data['Log_Returns'].std()
        annual_vol = daily_vol * np.sqrt(252)  # 252 trading days in a year
        
    elif method == 'parkinson':
        # Parkinson's volatility estimator (uses high-low prices)
        hl_ratio = np.log(data['High'] / data['Low'])
        annual_vol = np.sqrt(np.mean(hl_ratio**2) * 252 / (4 * np.log(2)))
        
    print(f"   - {method.title()} volatility: {annual_vol:.4f} ({annual_vol*100:.2f}%)")
    return annual_vol

In [4]:
def save_to_csv(data, filename):
    """Save data to CSV file for record keeping"""
    data.to_csv(filename)
    print(f"   - Data saved to {filename}")

In [5]:
# Fetch Reliance stock data
reliance_data = fetch_stock_data("RELIANCE.NS", "2y")

# Calculate volatility using different methods
print("\n2. Calculating Volatility Metrics...")
hist_vol = calculate_volatility(reliance_data, 'historical')
park_vol = calculate_volatility(reliance_data, 'parkinson') 


1. Fetching data for RELIANCE.NS...
   - Downloaded 494 trading days of data
   - Current stock price: ₹1476.00
   - Price range: ₹1109.48 - ₹1595.48

2. Calculating Volatility Metrics...
   - Historical volatility: 0.2107 (21.07%)
   - Parkinson volatility: 0.1831 (18.31%)


In [6]:
# Use historical volatility for our models
volatility = hist_vol

# Save data to CSV
save_to_csv(reliance_data, 'reliance_stock_data.csv')

# Current market parameters (based on research)
current_price = reliance_data['Close'].iloc[-1]
risk_free_rate = 0.0631  # 10Y Government Bond yield as of July 2025
print(f"\n3. Market Parameters:")
print(f"   - Current Stock Price (S₀): ₹{current_price:.2f}")
print(f"   - Historical Volatility (σ): {volatility:.4f}")
print(f"   - Risk-free Rate (r): {risk_free_rate:.4f}")


   - Data saved to reliance_stock_data.csv

3. Market Parameters:
   - Current Stock Price (S₀): ₹1476.00
   - Historical Volatility (σ): 0.2107
   - Risk-free Rate (r): 0.0631


## 3. Black-Scholes Model

In [7]:
class BlackScholesModel:
    """
    Black-Scholes option pricing model implementation
    
    This is the most famous option pricing model, developed by Fischer Black, 
    Myron Scholes, and Robert Merton. It provides a mathematical framework
    for determining the theoretical price of European-style options.
    
    Key Assumptions:
    - Constant volatility and risk-free rate
    - Log-normal distribution of stock prices
    - No dividends during option life
    - European exercise (only at expiry)
    """
    
    def __init__(self, S, K, T, r, sigma):
        """
        Initialize Black-Scholes parameters
        
        Parameters:
        S: Current stock price
        K: Strike price
        T: Time to expiry (in years)
        r: Risk-free rate
        sigma: Volatility
        """
        self.S = S      # Current stock price
        self.K = K      # Strike price  
        self.T = T      # Time to expiry
        self.r = r      # Risk-free rate
        self.sigma = sigma  # Volatility
    
    def d1(self):
        """Calculate d1 parameter used in Black-Scholes formula"""
        return (np.log(self.S / self.K) + (self.r + 0.5 * self.sigma**2) * self.T) / (self.sigma * np.sqrt(self.T))
    
    def d2(self):
        """Calculate d2 parameter used in Black-Scholes formula"""
        return self.d1() - self.sigma * np.sqrt(self.T)
    
    def call_price(self):
        """
        Calculate European call option price using Black-Scholes formula
        
        Formula: C = S₀ * N(d₁) - K * e^(-rT) * N(d₂)
        where N(x) is the cumulative standard normal distribution
        """
        d1_val = self.d1()
        d2_val = self.d2()
        
        call_value = (self.S * norm.cdf(d1_val) - 
                     self.K * np.exp(-self.r * self.T) * norm.cdf(d2_val))
        return call_value
    
    def put_price(self):
        """
        Calculate European put option price using put-call parity
        
        Formula: P = K * e^(-rT) * N(-d₂) - S₀ * N(-d₁)
        """
        d1_val = self.d1()
        d2_val = self.d2()
        
        put_value = (self.K * np.exp(-self.r * self.T) * norm.cdf(-d2_val) - 
                    self.S * norm.cdf(-d1_val))
        return put_value
    
    def greeks(self):
        """
        Calculate option Greeks (risk sensitivities)
        
        Greeks measure how option prices change with respect to various factors:
        - Delta: Price sensitivity to underlying stock price
        - Gamma: Rate of change of delta
        - Theta: Time decay
        - Vega: Volatility sensitivity
        - Rho: Interest rate sensitivity
        """
        d1_val = self.d1()
        d2_val = self.d2()
        
        # Delta (∂C/∂S): Rate of change of option price with respect to stock price
        call_delta = norm.cdf(d1_val)
        put_delta = call_delta - 1
        
        # Gamma (∂²C/∂S²): Rate of change of delta with respect to stock price
        gamma = norm.pdf(d1_val) / (self.S * self.sigma * np.sqrt(self.T))
        
        # Theta (∂C/∂T): Time decay of option value
        call_theta = (-(self.S * norm.pdf(d1_val) * self.sigma) / (2 * np.sqrt(self.T)) - 
                     self.r * self.K * np.exp(-self.r * self.T) * norm.cdf(d2_val)) / 365
        
        # Vega (∂C/∂σ): Sensitivity to volatility changes
        vega = self.S * norm.pdf(d1_val) * np.sqrt(self.T) / 100
        
        # Rho (∂C/∂r): Sensitivity to interest rate changes
        call_rho = self.K * self.T * np.exp(-self.r * self.T) * norm.cdf(d2_val) / 100
        
        return {
            'call_delta': call_delta, 'put_delta': put_delta,
            'gamma': gamma, 'call_theta': call_theta,
            'vega': vega, 'call_rho': call_rho
        }

In [8]:
print("\n" + "="*60)
print("BLACK-SCHOLES MODEL ANALYSIS")
print("="*60)

# Define option parameters for analysis
strikes = [1300, 1400, 1450, 1476, 1500, 1550, 1600]  # Strike prices around current price
time_to_expiry = 30/365  # 30 days to expiry (converted to years)

bs_results = []

print(f"\nBlack-Scholes Analysis for Reliance Options (30 days to expiry):")
print(f"Current Stock Price: ₹{current_price:.2f}")
print("-" * 80)
print(f"{'Strike':<8} {'Call Price':<12} {'Put Price':<12} {'Delta':<8} {'Gamma':<8} {'Vega':<8}")
print("-" * 80)

for strike in strikes:
    # Create Black-Scholes model instance
    bs_model = BlackScholesModel(current_price, strike, time_to_expiry, risk_free_rate, volatility)
    
    # Calculate option prices
    call_price = bs_model.call_price()
    put_price = bs_model.put_price()
    
    # Calculate Greeks
    greeks = bs_model.greeks()
    
    # Store results
    bs_results.append({
        'Strike': strike,
        'Call_Price': call_price,
        'Put_Price': put_price,
        'Call_Delta': greeks['call_delta'],
        'Gamma': greeks['gamma'],
        'Vega': greeks['vega'],
        'Moneyness': current_price / strike  # S/K ratio
    })
    
    # Print formatted results
    print(f"{strike:<8} ₹{call_price:<11.2f} ₹{put_price:<11.2f} {greeks['call_delta']:<7.3f} {greeks['gamma']:<7.4f} {greeks['vega']:<7.2f}")

# Convert to DataFrame for easier manipulation
bs_df = pd.DataFrame(bs_results)


BLACK-SCHOLES MODEL ANALYSIS

Black-Scholes Analysis for Reliance Options (30 days to expiry):
Current Stock Price: ₹1476.00
--------------------------------------------------------------------------------
Strike   Call Price   Put Price    Delta    Gamma    Vega    
--------------------------------------------------------------------------------
1300     ₹183.15      ₹0.42        0.987   0.0004  0.14   
1400     ₹91.01       ₹7.77        0.839   0.0027  1.03   
1450     ₹54.42       ₹20.92       0.659   0.0041  1.55   
1476     ₹39.43       ₹31.79       0.546   0.0044  1.68   
1500     ₹28.23       ₹44.47       0.440   0.0044  1.67   
1550     ₹12.50       ₹78.48       0.244   0.0035  1.33   
1600     ₹4.69        ₹120.41      0.111   0.0021  0.80   


## 4. Monte Carlo Simulation

In [9]:
class MonteCarloOptions:
    """
    Monte Carlo simulation for option pricing
    
    Monte Carlo methods use random sampling to solve numerical problems.
    For options, we simulate many possible price paths for the underlying stock
    and calculate the average payoff to determine the option's fair value.
    
    Advantages:
    - Can handle complex payoffs and exotic options
    - Provides confidence intervals
    - Flexible for various underlying processes
    
    Disadvantages:
    - Computationally intensive
    - Convergence can be slow
    - Requires large number of simulations for accuracy
    """
    
    def __init__(self, S0, K, T, r, sigma, n_sims=100000):
        """
        Initialize Monte Carlo parameters
        
        Parameters:
        S0: Initial stock price
        K: Strike price
        T: Time to expiry
        r: Risk-free rate
        sigma: Volatility
        n_sims: Number of simulation paths
        """
        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma
        self.n_sims = n_sims
    
    def simulate_prices(self):
        """
        Simulate stock price paths using Geometric Brownian Motion
        
        The stock price follows the stochastic differential equation:
        dS = μSdt + σSdW
        
        Where:
        - μ is the drift (risk-free rate under risk-neutral measure)
        - σ is the volatility
        - dW is a Wiener process (random walk)
        
        Solution: S(T) = S(0) * exp((r - σ²/2)T + σ√T * Z)
        where Z ~ N(0,1)
        """
        
        # Generate random numbers from standard normal distribution
        Z = np.random.standard_normal(self.n_sims)
        
        # Calculate stock prices at expiry using GBM formula
        ST = self.S0 * np.exp((self.r - 0.5 * self.sigma**2) * self.T + 
                              self.sigma * np.sqrt(self.T) * Z)
        
        return ST
    
    def price_european_call(self):
        """Price European call option using Monte Carlo"""
        # Simulate final stock prices
        ST = self.simulate_prices()
        
        # Calculate call payoffs: max(S_T - K, 0)
        call_payoffs = np.maximum(ST - self.K, 0)
        
        # Discount expected payoff to present value
        call_price = np.exp(-self.r * self.T) * np.mean(call_payoffs)
        
        # Calculate confidence interval (95%)
        payoff_std = np.std(call_payoffs)
        standard_error = payoff_std / np.sqrt(self.n_sims)
        confidence_interval = 1.96 * standard_error * np.exp(-self.r * self.T)
        
        return call_price, confidence_interval
    
    def price_european_put(self):
        """Price European put option using Monte Carlo"""
        # Simulate final stock prices
        ST = self.simulate_prices()
        
        # Calculate put payoffs: max(K - S_T, 0)
        put_payoffs = np.maximum(self.K - ST, 0)
        
        # Discount expected payoff to present value
        put_price = np.exp(-self.r * self.T) * np.mean(put_payoffs)
        
        # Calculate confidence interval (95%)
        payoff_std = np.std(put_payoffs)
        standard_error = payoff_std / np.sqrt(self.n_sims)
        confidence_interval = 1.96 * standard_error * np.exp(-self.r * self.T)
        
        return put_price, confidence_interval
    
    def asian_option_call(self, n_steps=50):
        """
        Price Asian call option (average price option) using Monte Carlo
        
        Asian options have payoffs based on the average price of the underlying
        over a specified period, making them less sensitive to price manipulation.
        """
        dt = self.T / n_steps
        prices = np.zeros((self.n_sims, n_steps + 1))
        prices[:, 0] = self.S0
        
        for i in range(1, n_steps + 1):
            Z = np.random.standard_normal(self.n_sims)
            prices[:, i] = prices[:, i-1] * np.exp((self.r - 0.5 * self.sigma**2) * dt + 
                                                  self.sigma * np.sqrt(dt) * Z)
        
        # Calculate average prices
        avg_prices = np.mean(prices, axis=1)
        
        # Calculate Asian call payoffs
        asian_payoffs = np.maximum(avg_prices - self.K, 0)
        asian_call_price = np.exp(-self.r * self.T) * np.mean(asian_payoffs)
        
        return asian_call_price

In [10]:
print("\n" + "="*60)
print("MONTE CARLO SIMULATION ANALYSIS")
print("="*60)

mc_results = []

print(f"\nMonte Carlo Analysis (100,000 simulations per option):")
print("-" * 70)
print(f"{'Strike':<8} {'MC Call':<12} {'BS Call':<12} {'MC Put':<12} {'BS Put':<12} {'95% CI':<12}")
print("-" * 70)

for i, strike in enumerate(strikes):
    # Create Monte Carlo model
    mc_model = MonteCarloOptions(current_price, strike, time_to_expiry, 
                                risk_free_rate, volatility, n_sims=100000)
    
    # Price options using Monte Carlo
    mc_call, call_ci = mc_model.price_european_call()
    mc_put, put_ci = mc_model.price_european_put()
    
    # Get corresponding Black-Scholes prices for comparison
    bs_call = bs_results[i]['Call_Price']
    bs_put = bs_results[i]['Put_Price']
    
    # Store results
    mc_results.append({
        'Strike': strike,
        'MC_Call': mc_call,
        'MC_Put': mc_put,
        'BS_Call': bs_call,
        'BS_Put': bs_put,
        'Call_CI': call_ci,
        'Put_CI': put_ci,
        'Call_Diff': abs(mc_call - bs_call),
        'Put_Diff': abs(mc_put - bs_put)
    })
    
    print(f"{strike:<8} ₹{mc_call:<11.2f} ₹{bs_call:<11.2f} ₹{mc_put:<11.2f} ₹{bs_put:<11.2f} ±₹{call_ci:<10.2f}")

# Calculate overall accuracy metrics
mc_df = pd.DataFrame(mc_results)
avg_call_error = mc_df['Call_Diff'].mean()
avg_put_error = mc_df['Put_Diff'].mean()

print(f"\nMonte Carlo vs Black-Scholes Comparison:")
print(f"Average Call Price Difference: ₹{avg_call_error:.3f}")
print(f"Average Put Price Difference: ₹{avg_put_error:.3f}")
print(f"Maximum Call Difference: ₹{mc_df['Call_Diff'].max():.3f}")


MONTE CARLO SIMULATION ANALYSIS

Monte Carlo Analysis (100,000 simulations per option):
----------------------------------------------------------------------
Strike   MC Call      BS Call      MC Put       BS Put       95% CI      
----------------------------------------------------------------------
1300     ₹183.24      ₹183.15      ₹0.41        ₹0.42        ±₹0.55      
1400     ₹90.86       ₹91.01       ₹7.79        ₹7.77        ±₹0.48      
1450     ₹54.11       ₹54.42       ₹21.06       ₹20.92       ±₹0.40      
1476     ₹39.57       ₹39.43       ₹31.73       ₹31.79       ±₹0.35      
1500     ₹28.12       ₹28.23       ₹44.86       ₹44.47       ±₹0.30      
1550     ₹12.68       ₹12.50       ₹78.44       ₹78.48       ±₹0.20      
1600     ₹4.59        ₹4.69        ₹120.58      ₹120.41      ±₹0.12      

Monte Carlo vs Black-Scholes Comparison:
Average Call Price Difference: ₹0.155
Average Put Price Difference: ₹0.118
Maximum Call Difference: ₹0.313


## 5. Binomial Tree Model

In [11]:
class BinomialTreeModel:
    """
    Binomial Tree model for option pricing
    
    The binomial model creates a discrete-time framework where the stock price
    can move up or down by specific factors at each time step. This creates
    a binary tree of possible price paths.
    
    Advantages:
    - Can price American options (early exercise)
    - Intuitive and transparent methodology
    - Converges to Black-Scholes as steps increase
    
    Key Parameters:
    - u: Up factor (how much price increases)
    - d: Down factor (how much price decreases)  
    - p: Risk-neutral probability of up move
    """
    
    def __init__(self, S0, K, T, r, sigma, n_steps=100):
        """
        Initialize Binomial Tree parameters
        
        Parameters:
        S0: Initial stock price
        K: Strike price
        T: Time to expiry
        r: Risk-free rate
        sigma: Volatility
        n_steps: Number of time steps in the tree
        """
        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma
        self.n_steps = n_steps
        
        # Calculate time step
        self.dt = T / n_steps
        
        # Calculate binomial parameters using Cox-Ross-Rubinstein method
        self.u = np.exp(sigma * np.sqrt(self.dt))      # Up factor
        self.d = 1 / self.u                            # Down factor
        self.p = (np.exp(r * self.dt) - self.d) / (self.u - self.d)  # Risk-neutral probability
        
        # Discount factor
        self.discount = np.exp(-r * self.dt)
    
    def build_stock_tree(self):
        """
        Build the stock price tree
        
        Creates a 2D array where stock_tree[i][j] represents the stock price
        at time step i and node j
        """
        stock_tree = np.zeros((self.n_steps + 1, self.n_steps + 1))
        
        # Fill the tree
        for i in range(self.n_steps + 1):
            for j in range(i + 1):
                # Price = S0 * u^j * d^(i-j)
                stock_tree[i][j] = self.S0 * (self.u ** j) * (self.d ** (i - j))
        
        return stock_tree
    
    def european_call(self):
        """Price European call option using binomial tree"""
        # Build stock price tree
        stock_tree = self.build_stock_tree()
        
        # Initialize option value tree
        option_tree = np.zeros((self.n_steps + 1, self.n_steps + 1))
        
        # Calculate option values at expiry (final time step)
        for j in range(self.n_steps + 1):
            option_tree[self.n_steps][j] = max(stock_tree[self.n_steps][j] - self.K, 0)
        
        # Work backwards through the tree
        for i in range(self.n_steps - 1, -1, -1):
            for j in range(i + 1):
                # Option value = discounted expected value of next period
                option_tree[i][j] = self.discount * (
                    self.p * option_tree[i + 1][j + 1] + 
                    (1 - self.p) * option_tree[i + 1][j]
                )
        
        return option_tree[0][0]
    
    def european_put(self):
        """Price European put option using binomial tree"""
        # Build stock price tree
        stock_tree = self.build_stock_tree()
        
        # Initialize option value tree
        option_tree = np.zeros((self.n_steps + 1, self.n_steps + 1))
        
        # Calculate option values at expiry
        for j in range(self.n_steps + 1):
            option_tree[self.n_steps][j] = max(self.K - stock_tree[self.n_steps][j], 0)
        
        # Work backwards through the tree
        for i in range(self.n_steps - 1, -1, -1):
            for j in range(i + 1):
                option_tree[i][j] = self.discount * (
                    self.p * option_tree[i + 1][j + 1] + 
                    (1 - self.p) * option_tree[i + 1][j]
                )
        
        return option_tree[0][0]
    
    def american_call(self):
        """
        Price American call option with early exercise feature
        
        American options can be exercised at any time before expiry,
        so we need to check at each node whether immediate exercise
        is more valuable than holding the option.
        """
        stock_tree = self.build_stock_tree()
        option_tree = np.zeros((self.n_steps + 1, self.n_steps + 1))
        
        # Calculate option values at expiry
        for j in range(self.n_steps + 1):
            option_tree[self.n_steps][j] = max(stock_tree[self.n_steps][j] - self.K, 0)
        
        # Work backwards, checking for early exercise at each node
        for i in range(self.n_steps - 1, -1, -1):
            for j in range(i + 1):
                # Calculate holding value (discounted expected value)
                holding_value = self.discount * (
                    self.p * option_tree[i + 1][j + 1] + 
                    (1 - self.p) * option_tree[i + 1][j]
                )
                
                # Calculate immediate exercise value
                exercise_value = max(stock_tree[i][j] - self.K, 0)
                
                # Take maximum of holding and exercising
                option_tree[i][j] = max(holding_value, exercise_value)
        
        return option_tree[0][0]
    
    def american_put(self):
        """Price American put option with early exercise feature"""
        stock_tree = self.build_stock_tree()
        option_tree = np.zeros((self.n_steps + 1, self.n_steps + 1))
        
        # Calculate option values at expiry
        for j in range(self.n_steps + 1):
            option_tree[self.n_steps][j] = max(self.K - stock_tree[self.n_steps][j], 0)
        
        # Work backwards, checking for early exercise
        for i in range(self.n_steps - 1, -1, -1):
            for j in range(i + 1):
                # Holding value
                holding_value = self.discount * (
                    self.p * option_tree[i + 1][j + 1] + 
                    (1 - self.p) * option_tree[i + 1][j]
                )
                
                # Exercise value
                exercise_value = max(self.K - stock_tree[i][j], 0)
                
                # American feature: can exercise early
                option_tree[i][j] = max(holding_value, exercise_value)
        
        return option_tree[0][0]

In [12]:
print("\n" + "="*60)
print("BINOMIAL TREE MODEL ANALYSIS")
print("="*60)

bt_results = []

print(f"\nBinomial Tree Analysis (100 time steps):")
print("-" * 90)
print(f"{'Strike':<8} {'BT Call':<10} {'BS Call':<10} {'BT Put':<10} {'BS Put':<10} {'Am Call':<10} {'Am Put':<10}")
print("-" * 90)

for i, strike in enumerate(strikes):
    # Create Binomial Tree model
    bt_model = BinomialTreeModel(current_price, strike, time_to_expiry, 
                                risk_free_rate, volatility, n_steps=100)
    
    # Calculate European option prices
    bt_call = bt_model.european_call()
    bt_put = bt_model.european_put()
    
    # Calculate American option prices
    am_call = bt_model.american_call()
    am_put = bt_model.american_put()
    
    # Get Black-Scholes prices for comparison
    bs_call = bs_results[i]['Call_Price']
    bs_put = bs_results[i]['Put_Price']
    
    # Store results
    bt_results.append({
        'Strike': strike,
        'BT_Call': bt_call,
        'BT_Put': bt_put,
        'BS_Call': bs_call,
        'BS_Put': bs_put,
        'American_Call': am_call,
        'American_Put': am_put,
        'Early_Exercise_Premium_Call': am_call - bt_call,
        'Early_Exercise_Premium_Put': am_put - bt_put
    })
    
    print(f"{strike:<8} ₹{bt_call:<9.2f} ₹{bs_call:<9.2f} ₹{bt_put:<9.2f} ₹{bs_put:<9.2f} ₹{am_call:<9.2f} ₹{am_put:<9.2f}")

bt_df = pd.DataFrame(bt_results)

# Analyze early exercise premiums
avg_call_premium = bt_df['Early_Exercise_Premium_Call'].mean()
avg_put_premium = bt_df['Early_Exercise_Premium_Put'].mean()

print(f"\nEarly Exercise Analysis:")
print(f"Average Call Early Exercise Premium: ₹{avg_call_premium:.3f}")
print(f"Average Put Early Exercise Premium: ₹{avg_put_premium:.3f}")


BINOMIAL TREE MODEL ANALYSIS

Binomial Tree Analysis (100 time steps):
------------------------------------------------------------------------------------------
Strike   BT Call    BS Call    BT Put     BS Put     Am Call    Am Put    
------------------------------------------------------------------------------------------
1300     ₹183.14    ₹183.15    ₹0.42      ₹0.42      ₹183.14    ₹0.42     
1400     ₹91.05     ₹91.01     ₹7.80      ₹7.77      ₹91.05     ₹7.90     
1450     ₹54.50     ₹54.42     ₹21.00     ₹20.92     ₹54.50     ₹21.34    
1476     ₹39.34     ₹39.43     ₹31.70     ₹31.79     ₹39.34     ₹32.32    
1500     ₹28.29     ₹28.23     ₹44.53     ₹44.47     ₹28.29     ₹45.48    
1550     ₹12.44     ₹12.50     ₹78.42     ₹78.48     ₹12.44     ₹80.59    
1600     ₹4.69      ₹4.69      ₹120.42    ₹120.41    ₹4.69      ₹124.65   

Early Exercise Analysis:
Average Call Early Exercise Premium: ₹0.000
Average Put Early Exercise Premium: ₹1.202


## 6. XGBoost Model

In [13]:
class XGBoostOptionPricer:
    """
    XGBoost machine learning model for option pricing
    
    This approach uses gradient boosting to learn the relationship between
    option parameters and option prices. It can capture non-linear patterns
    and market inefficiencies that traditional models might miss.
    
    Features used:
    - Moneyness (S/K ratio)
    - Time to expiry
    - Volatility
    - Interest rate
    - Historical price momentum
    - Volatility smile effects
    """
    
    def __init__(self):
        self.model_call = None
        self.model_put = None
        self.feature_names = ['moneyness', 'time_to_expiry', 'volatility', 
                             'risk_free_rate', 'price_momentum', 'vol_smile']
    
    def generate_training_data(self, n_samples=10000):
        """
        Generate synthetic training data using Black-Scholes as baseline
        and add market noise and smile effects to create realistic training set
        
        This approach simulates various market conditions and option parameters
        to create a comprehensive training dataset
        """
        print("\nGenerating training data for XGBoost model...")
        
        np.random.seed(42)
        
        # Generate random option parameters
        S_range = np.random.uniform(1200, 1800, n_samples)  # Stock price range around Reliance
        K_range = np.random.uniform(1200, 1800, n_samples)  # Strike price range
        T_range = np.random.uniform(0.02, 0.25, n_samples)  # 1 week to 3 months
        vol_range = np.random.uniform(0.15, 0.45, n_samples)  # Volatility range
        r_range = np.random.uniform(0.05, 0.08, n_samples)  # Interest rate range
        
        features = []
        call_targets = []
        put_targets = []
        
        for i in range(n_samples):
            S, K, T, vol, r = S_range[i], K_range[i], T_range[i], vol_range[i], r_range[i]
            
            # Calculate moneyness
            moneyness = S / K
            
            # Add momentum feature (simulated)
            momentum = np.random.normal(0, 0.1)
            
            # Add volatility smile effect (higher vol for OTM options)
            if moneyness < 0.95 or moneyness > 1.05:
                vol_smile = vol * (1 + 0.1 * abs(1 - moneyness))
            else:
                vol_smile = vol
            
            # Calculate base Black-Scholes price
            bs_temp = BlackScholesModel(S, K, T, r, vol_smile)
            bs_call = bs_temp.call_price()
            bs_put = bs_temp.put_price()
            
            # Add market noise (bid-ask spread, liquidity effects)
            noise_call = np.random.normal(0, bs_call * 0.02)  # 2% noise
            noise_put = np.random.normal(0, bs_put * 0.02)
            
            # Store features and targets
            features.append([moneyness, T, vol, r, momentum, vol_smile])
            call_targets.append(max(bs_call + noise_call, 0))
            put_targets.append(max(bs_put + noise_put, 0))
        
        return np.array(features), np.array(call_targets), np.array(put_targets)
    
    def train_model(self, n_samples=10000):
        """Train XGBoost models for call and put options"""
        # Generate training data
        X, y_call, y_put = self.generate_training_data(n_samples)
        
        # Split into training and validation sets
        X_train, X_val, y_call_train, y_call_val = train_test_split(X, y_call, test_size=0.2, random_state=42)
        _, _, y_put_train, y_put_val = train_test_split(X, y_put, test_size=0.2, random_state=42)
        
        # XGBoost parameters optimized for option pricing
        xgb_params = {
            'objective': 'reg:squarederror',
            'max_depth': 6,
            'learning_rate': 0.1,
            'n_estimators': 200,
            'subsample': 0.8,
            'colsample_bytree': 0.8,
            'random_state': 42
        }
        
        # Train call option model
        print("Training call option model...")
        self.model_call = xgb.XGBRegressor(**xgb_params)
        self.model_call.fit(X_train, y_call_train,
                           eval_set=[(X_val, y_call_val)],
                           verbose=False)
        
        # Train put option model
        print("Training put option model...")
        self.model_put = xgb.XGBRegressor(**xgb_params)
        self.model_put.fit(X_train, y_put_train,
                          eval_set=[(X_val, y_put_val)],
                          verbose=False)
        
        # Calculate training metrics
        call_pred_val = self.model_call.predict(X_val)
        put_pred_val = self.model_put.predict(X_val)
        
        call_mse = mean_squared_error(y_call_val, call_pred_val)
        put_mse = mean_squared_error(y_put_val, put_pred_val)
        call_mae = mean_absolute_error(y_call_val, call_pred_val)
        put_mae = mean_absolute_error(y_put_val, put_pred_val)
        
        print(f"Call Model - MSE: {call_mse:.4f}, MAE: {call_mae:.4f}")
        print(f"Put Model - MSE: {put_mse:.4f}, MAE: {put_mae:.4f}")
        
        return X_val, y_call_val, y_put_val, call_pred_val, put_pred_val
    
    def predict_option_price(self, S, K, T, r, sigma):
        """Predict option prices using trained XGBoost models"""
        moneyness = S / K
        momentum = 0  # Assume neutral momentum for prediction
        
        # Apply volatility smile effect
        if moneyness < 0.95 or moneyness > 1.05:
            vol_smile = sigma * (1 + 0.1 * abs(1 - moneyness))
        else:
            vol_smile = sigma
        
        # Prepare features
        features = np.array([[moneyness, T, sigma, r, momentum, vol_smile]])
        
        # Make predictions
        call_price = self.model_call.predict(features)[0]
        put_price = self.model_put.predict(features)[0]
        
        return max(call_price, 0), max(put_price, 0)
    
    def feature_importance_analysis(self):
        """Analyze which features are most important for pricing"""
        if self.model_call is None:
            print("Model not trained yet!")
            return
        
        # Get feature importances
        call_importance = self.model_call.feature_importances_
        put_importance = self.model_put.feature_importances_
        
        print("\nFeature Importance Analysis:")
        print("-" * 50)
        print(f"{'Feature':<20} {'Call Model':<12} {'Put Model':<12}")
        print("-" * 50)
        
        for i, feature in enumerate(self.feature_names):
            print(f"{feature:<20} {call_importance[i]:<11.3f} {put_importance[i]:<11.3f}")

In [14]:
print("\n" + "="*60)
print("XGBOOST MACHINE LEARNING MODEL")
print("="*60)

# Initialize and train XGBoost model
xgb_pricer = XGBoostOptionPricer()
X_val, y_call_val, y_put_val, call_pred_val, put_pred_val = xgb_pricer.train_model(n_samples=15000)

# Feature importance analysis
xgb_pricer.feature_importance_analysis()

# Test XGBoost model on our strike prices
xgb_results = []

print(f"\nXGBoost vs Other Models Comparison:")
print("-" * 90)
print(f"{'Strike':<8} {'XGB Call':<10} {'BS Call':<10} {'XGB Put':<10} {'BS Put':<10} {'Call Diff':<10} {'Put Diff':<10}")
print("-" * 90)

for i, strike in enumerate(strikes):
    # Get XGBoost predictions
    xgb_call, xgb_put = xgb_pricer.predict_option_price(
        current_price, strike, time_to_expiry, risk_free_rate, volatility
    )
    
    # Get Black-Scholes prices for comparison
    bs_call = bs_results[i]['Call_Price']
    bs_put = bs_results[i]['Put_Price']
    
    # Calculate differences
    call_diff = abs(xgb_call - bs_call)
    put_diff = abs(xgb_put - bs_put)
    
    # Store results
    xgb_results.append({
        'Strike': strike,
        'XGB_Call': xgb_call,
        'XGB_Put': xgb_put,
        'BS_Call': bs_call,
        'BS_Put': bs_put,
        'Call_Diff': call_diff,
        'Put_Diff': put_diff
    })
    
    print(f"{strike:<8} ₹{xgb_call:<9.2f} ₹{bs_call:<9.2f} ₹{xgb_put:<9.2f} ₹{bs_put:<9.2f} ₹{call_diff:<9.3f} ₹{put_diff:<9.3f}")

xgb_df = pd.DataFrame(xgb_results)


XGBOOST MACHINE LEARNING MODEL

Generating training data for XGBoost model...
Training call option model...
Training put option model...
Call Model - MSE: 165.0452, MAE: 8.9274
Put Model - MSE: 135.0043, MAE: 8.0900

Feature Importance Analysis:
--------------------------------------------------
Feature              Call Model   Put Model   
--------------------------------------------------
moneyness            0.940       0.956      
time_to_expiry       0.015       0.009      
volatility           0.016       0.009      
risk_free_rate       0.005       0.006      
price_momentum       0.004       0.005      
vol_smile            0.020       0.014      

XGBoost vs Other Models Comparison:
------------------------------------------------------------------------------------------
Strike   XGB Call   BS Call    XGB Put    BS Put     Call Diff  Put Diff  
------------------------------------------------------------------------------------------
1300     ₹199.11    ₹183.15    ₹0.61    

## 7. Model Comparison

In [16]:
# Create comprehensive comparison DataFrame
comparison_data = []

for i, strike in enumerate(strikes):
    comparison_data.append({
        'Strike': strike,
        'Moneyness': current_price / strike,
        'Black_Scholes_Call': bs_results[i]['Call_Price'],
        'Black_Scholes_Put': bs_results[i]['Put_Price'],
        'Monte_Carlo_Call': mc_results[i]['MC_Call'],
        'Monte_Carlo_Put': mc_results[i]['MC_Put'],
        'Binomial_Tree_Call': bt_results[i]['BT_Call'],
        'Binomial_Tree_Put': bt_results[i]['BT_Put'],
        'XGBoost_Call': xgb_results[i]['XGB_Call'],
        'XGBoost_Put': xgb_results[i]['XGB_Put'],
        'American_Call': bt_results[i]['American_Call'],
        'American_Put': bt_results[i]['American_Put']
    })

comparison_df = pd.DataFrame(comparison_data)

# Save comprehensive results to CSV
comparison_df.to_csv('options_pricing_comparison.csv', index=False)
print("\nDetailed comparison saved to: options_pricing_comparison.csv")


Detailed comparison saved to: options_pricing_comparison.csv


In [17]:
# Statistical Analysis
print("\nStatistical Analysis of Model Performance:")
print("=" * 60)

# Calculate correlations between models
call_correlations = comparison_df[['Black_Scholes_Call', 'Monte_Carlo_Call', 
                                   'Binomial_Tree_Call', 'XGBoost_Call']].corr()
put_correlations = comparison_df[['Black_Scholes_Put', 'Monte_Carlo_Put', 
                                  'Binomial_Tree_Put', 'XGBoost_Put']].corr()

print("\nCall Options - Model Correlations:")
print(call_correlations.round(4))

print("\nPut Options - Model Correlations:")
print(put_correlations.round(4))

# Performance metrics relative to Black-Scholes (as benchmark)
print("\nModel Performance vs Black-Scholes (RMSE and MAE):")
print("-" * 60)

models = ['Monte_Carlo', 'Binomial_Tree', 'XGBoost']
for model in models:
    # Call options
    call_diff = comparison_df[f'{model}_Call'] - comparison_df['Black_Scholes_Call']
    call_rmse = np.sqrt(np.mean(call_diff**2))
    call_mae = np.mean(np.abs(call_diff))
    
    # Put options
    put_diff = comparison_df[f'{model}_Put'] - comparison_df['Black_Scholes_Put']
    put_rmse = np.sqrt(np.mean(put_diff**2))
    put_mae = np.mean(np.abs(put_diff))
    
    print(f"{model:<15} Call RMSE: ₹{call_rmse:.3f}, MAE: ₹{call_mae:.3f}")
    print(f"{'':<15} Put RMSE:  ₹{put_rmse:.3f}, MAE: ₹{put_mae:.3f}")

# Moneyness analysis
print("\nPerformance by Moneyness Categories:")
print("-" * 50)

comparison_df['Moneyness_Category'] = pd.cut(comparison_df['Moneyness'], 
                                            bins=[0, 0.95, 1.05, float('inf')], 
                                            labels=['OTM', 'ATM', 'ITM'])

for category in ['OTM', 'ATM', 'ITM']:
    category_data = comparison_df[comparison_df['Moneyness_Category'] == category]
    if len(category_data) > 0:
        print(f"\n{category} Options:")
        
        # Calculate average prices by model for this category
        bs_avg = category_data['Black_Scholes_Call'].mean()
        mc_avg = category_data['Monte_Carlo_Call'].mean()
        bt_avg = category_data['Binomial_Tree_Call'].mean()
        xgb_avg = category_data['XGBoost_Call'].mean()
        
        print(f"  Average Call Price - BS: ₹{bs_avg:.2f}, MC: ₹{mc_avg:.2f}, BT: ₹{bt_avg:.2f}, XGB: ₹{xgb_avg:.2f}")



Statistical Analysis of Model Performance:

Call Options - Model Correlations:
                    Black_Scholes_Call  Monte_Carlo_Call  Binomial_Tree_Call  \
Black_Scholes_Call              1.0000            1.0000              1.0000   
Monte_Carlo_Call                1.0000            1.0000              1.0000   
Binomial_Tree_Call              1.0000            1.0000              1.0000   
XGBoost_Call                    0.9996            0.9996              0.9995   

                    XGBoost_Call  
Black_Scholes_Call        0.9996  
Monte_Carlo_Call          0.9996  
Binomial_Tree_Call        0.9995  
XGBoost_Call              1.0000  

Put Options - Model Correlations:
                   Black_Scholes_Put  Monte_Carlo_Put  Binomial_Tree_Put  \
Black_Scholes_Put             1.0000           1.0000             1.0000   
Monte_Carlo_Put               1.0000           1.0000             1.0000   
Binomial_Tree_Put             1.0000           1.0000             1.0000   
XGBoo

## 8. Sensitivity Analysis and Greeks Comparison

In [18]:
def sensitivity_analysis():
    """
    Perform sensitivity analysis to understand how each model responds
    to changes in underlying parameters
    """
    
    base_strike = 1476  # ATM strike close to current price
    
    # Parameter ranges for sensitivity analysis
    spot_range = np.linspace(1300, 1650, 20)
    vol_range = np.linspace(0.15, 0.50, 15)
    time_range = np.linspace(0.01, 0.25, 10)  # 1 week to 3 months
    
    sensitivity_results = {
        'spot_sensitivity': {'spot': [], 'bs_call': [], 'mc_call': [], 'bt_call': [], 'xgb_call': []},
        'vol_sensitivity': {'vol': [], 'bs_call': [], 'mc_call': [], 'bt_call': [], 'xgb_call': []},
        'time_sensitivity': {'time': [], 'bs_call': [], 'mc_call': [], 'bt_call': [], 'xgb_call': []}
    }
    
    print("Performing sensitivity analysis...")
    
    # Spot price sensitivity (Delta equivalent)
    print("1. Spot price sensitivity analysis...")
    for spot in spot_range:
        # Black-Scholes
        bs_model = BlackScholesModel(spot, base_strike, time_to_expiry, risk_free_rate, volatility)
        bs_call = bs_model.call_price()
        
        # Monte Carlo (reduced simulations for speed)
        mc_model = MonteCarloOptions(spot, base_strike, time_to_expiry, risk_free_rate, volatility, 10000)
        mc_call, _ = mc_model.price_european_call()
        
        # Binomial Tree
        bt_model = BinomialTreeModel(spot, base_strike, time_to_expiry, risk_free_rate, volatility, 50)
        bt_call = bt_model.european_call()
        
        # XGBoost
        xgb_call, _ = xgb_pricer.predict_option_price(spot, base_strike, time_to_expiry, risk_free_rate, volatility)
        
        sensitivity_results['spot_sensitivity']['spot'].append(spot)
        sensitivity_results['spot_sensitivity']['bs_call'].append(bs_call)
        sensitivity_results['spot_sensitivity']['mc_call'].append(mc_call)
        sensitivity_results['spot_sensitivity']['bt_call'].append(bt_call)
        sensitivity_results['spot_sensitivity']['xgb_call'].append(xgb_call)
    
    # Volatility sensitivity (Vega equivalent)
    print("2. Volatility sensitivity analysis...")
    for vol in vol_range:
        bs_model = BlackScholesModel(current_price, base_strike, time_to_expiry, risk_free_rate, vol)
        bs_call = bs_model.call_price()
        
        mc_model = MonteCarloOptions(current_price, base_strike, time_to_expiry, risk_free_rate, vol, 10000)
        mc_call, _ = mc_model.price_european_call()
        
        bt_model = BinomialTreeModel(current_price, base_strike, time_to_expiry, risk_free_rate, vol, 50)
        bt_call = bt_model.european_call()
        
        xgb_call, _ = xgb_pricer.predict_option_price(current_price, base_strike, time_to_expiry, risk_free_rate, vol)
        
        sensitivity_results['vol_sensitivity']['vol'].append(vol)
        sensitivity_results['vol_sensitivity']['bs_call'].append(bs_call)
        sensitivity_results['vol_sensitivity']['mc_call'].append(mc_call)
        sensitivity_results['vol_sensitivity']['bt_call'].append(bt_call)
        sensitivity_results['vol_sensitivity']['xgb_call'].append(xgb_call)
    
    # Time decay sensitivity (Theta equivalent)
    print("3. Time decay sensitivity analysis...")
    for time_val in time_range:
        bs_model = BlackScholesModel(current_price, base_strike, time_val, risk_free_rate, volatility)
        bs_call = bs_model.call_price()
        
        mc_model = MonteCarloOptions(current_price, base_strike, time_val, risk_free_rate, volatility, 10000)
        mc_call, _ = mc_model.price_european_call()
        
        bt_model = BinomialTreeModel(current_price, base_strike, time_val, risk_free_rate, volatility, 50)
        bt_call = bt_model.european_call()
        
        xgb_call, _ = xgb_pricer.predict_option_price(current_price, base_strike, time_val, risk_free_rate, volatility)
        
        sensitivity_results['time_sensitivity']['time'].append(time_val)
        sensitivity_results['time_sensitivity']['bs_call'].append(bs_call)
        sensitivity_results['time_sensitivity']['mc_call'].append(mc_call)
        sensitivity_results['time_sensitivity']['bt_call'].append(bt_call)
        sensitivity_results['time_sensitivity']['xgb_call'].append(xgb_call)
    
    return sensitivity_results

In [19]:
# Perform sensitivity analysis
sensitivity_data = sensitivity_analysis()

# Save sensitivity analysis results
for analysis_type, data in sensitivity_data.items():
    df = pd.DataFrame(data)
    df.to_csv(f'{analysis_type}_analysis.csv', index=False)
    print(f"Saved {analysis_type} results to CSV")

Performing sensitivity analysis...
1. Spot price sensitivity analysis...
2. Volatility sensitivity analysis...
3. Time decay sensitivity analysis...
Saved spot_sensitivity results to CSV
Saved vol_sensitivity results to CSV
Saved time_sensitivity results to CSV


## 9. Final Summary

In [20]:
print("\n" + "="*80)
print("FINAL SUMMARY AND MODEL RECOMMENDATIONS")
print("="*80)

print("\nModel Performance Summary:")
print("-" * 40)

# Calculate final accuracy metrics
print(f"\nAccuracy Metrics (vs Black-Scholes baseline):")
print(f"Monte Carlo Call RMSE: ₹{np.sqrt(np.mean([(mc['MC_Call'] - mc['BS_Call'])**2 for mc in mc_results])):.3f}")
print(f"Binomial Tree Call RMSE: ₹{np.sqrt(np.mean([(bt['BT_Call'] - bt['BS_Call'])**2 for bt in bt_results])):.3f}")
print(f"XGBoost Call RMSE: ₹{np.sqrt(np.mean([xgb['Call_Diff']**2 for xgb in xgb_results])):.3f}")


FINAL SUMMARY AND MODEL RECOMMENDATIONS

Model Performance Summary:
----------------------------------------

Accuracy Metrics (vs Black-Scholes baseline):
Monte Carlo Call RMSE: ₹0.170
Binomial Tree Call RMSE: ₹0.058
XGBoost Call RMSE: ₹6.729


In [21]:
print(f"\n" + "="*80)
print("ANALYSIS COMPLETE - All data saved to CSV files")
print("="*80)

# Create final summary report
final_summary = {
    'Current_Stock_Price': current_price,
    'Volatility': volatility,
    'Risk_Free_Rate': risk_free_rate,
    'Analysis_Date': '2025-07-19',
    'Models_Compared': 4,
    'Strike_Prices_Analyzed': len(strikes),
    'ATM_Strike': min(strikes, key=lambda x: abs(x - current_price))
}

summary_df = pd.DataFrame([final_summary])
summary_df.to_csv('analysis_summary.csv', index=False)


ANALYSIS COMPLETE - All data saved to CSV files


In [22]:
print("1. BLACK-SCHOLES MODEL:")
print("   ✓ Pros: Fast computation, analytical solution, well-established")
print("   ✓ Greeks calculation built-in")
print("   ✗ Cons: Assumes constant volatility, European exercise only")
print("   📊 Best for: Quick pricing, risk management, liquid markets")

print("\n2. MONTE CARLO SIMULATION:")
print("   ✓ Pros: Handles complex payoffs, provides confidence intervals")
print("   ✓ Very flexible for exotic options")
print("   ✗ Cons: Computationally intensive, convergence issues")
print("   📊 Best for: Complex derivatives, risk scenario analysis")

print("\n3. BINOMIAL TREE MODEL:")
print("   ✓ Pros: American options, intuitive approach, converges to BS")
print("   ✓ Can handle dividends and early exercise")
print("   ✗ Cons: Moderate computational cost, discrete approximation")
print("   📊 Best for: American options, educational purposes")

print("\n4. XGBOOST MACHINE LEARNING:")
print("   ✓ Pros: Captures market inefficiencies, adapts to patterns")
print("   ✓ Can incorporate additional market factors")
print("   ✗ Cons: Black box, requires extensive training data")
print("   📊 Best for: Market-making, capturing smile effects")


1. BLACK-SCHOLES MODEL:
   ✓ Pros: Fast computation, analytical solution, well-established
   ✓ Greeks calculation built-in
   ✗ Cons: Assumes constant volatility, European exercise only
   📊 Best for: Quick pricing, risk management, liquid markets

2. MONTE CARLO SIMULATION:
   ✓ Pros: Handles complex payoffs, provides confidence intervals
   ✓ Very flexible for exotic options
   ✗ Cons: Computationally intensive, convergence issues
   📊 Best for: Complex derivatives, risk scenario analysis

3. BINOMIAL TREE MODEL:
   ✓ Pros: American options, intuitive approach, converges to BS
   ✓ Can handle dividends and early exercise
   ✗ Cons: Moderate computational cost, discrete approximation
   📊 Best for: American options, educational purposes

4. XGBOOST MACHINE LEARNING:
   ✓ Pros: Captures market inefficiencies, adapts to patterns
   ✓ Can incorporate additional market factors
   ✗ Cons: Black box, requires extensive training data
   📊 Best for: Market-making, capturing smile effects


In [23]:
print(f"\nComputational Efficiency Ranking:")
print("1. Black-Scholes (Fastest - analytical solution)")
print("2. Binomial Tree (Fast - O(n²) complexity)")
print("3. XGBoost (Moderate - after training)")
print("4. Monte Carlo (Slowest - depends on simulation count)")

print(f"\nRecommendations for Reliance Options Trading:")
print("1. Use Black-Scholes for quick pricing and delta hedging")
print("2. Use Binomial Tree for American-style options")
print("3. Use Monte Carlo for exotic payoffs and risk analysis")
print("4. Use XGBoost to capture market inefficiencies and smile effects")


Computational Efficiency Ranking:
1. Black-Scholes (Fastest - analytical solution)
2. Binomial Tree (Fast - O(n²) complexity)
3. XGBoost (Moderate - after training)
4. Monte Carlo (Slowest - depends on simulation count)

Recommendations for Reliance Options Trading:
1. Use Black-Scholes for quick pricing and delta hedging
2. Use Binomial Tree for American-style options
3. Use Monte Carlo for exotic payoffs and risk analysis
4. Use XGBoost to capture market inefficiencies and smile effects
